<a href="https://colab.research.google.com/github/yydevelop/study/blob/main/01_nlp_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「自然言語処理」 -コーパスの解析-
ライブラリPyCaretを使い、AutoMLによる「自然言語処理」を行います。  
クラウドファンディングによる融資に関する文章を解析し、返済の可能性を判定しましょう。  
まずは、PyCaretによる文章の解析から始めます。

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [1]:
!pip install pycaret==2.3.1

     |████████████████████████████████| 261 kB 4.1 MB/s 
     |████████████████████████████████| 1.7 MB 30.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 15.5 MB 18.3 MB/s 
     |████████████████████████████████| 167 kB 57.1 MB/s 
     |████████████████████████████████| 261 kB 66.1 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.8 MB 43.7 MB/s 
     |████████████████████████████████| 1.3 MB 26.1 MB/s 
     |████████████████████████████████| 2.0 MB 46.6 MB/s 
     |████████████████████████████████| 113 kB 60.7 MB/s 
     |████████████████████████████████| 102 kB 11.2 MB/s 
     |████████████████████████████████| 675 kB 60.1 MB/s 
     |████████████████████████████████| 303 kB 59.7 MB/s 
     |█████████████████

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [2]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


## データセットの読み込み
今回は、「Kiva Microfunds」のデータを使用します。  
Kiva Microfundsは、クラウドファンディングにより個人が世界中の低所得の起業家や学生に融資することを可能にする非営利団体です。  
Kivaの各融資のデータには、性別や所在地などの情報に加え、借り手のストーリーも含まれています。  
今回は、自然言語処理によりこの借り手のストーリの文章を解析します。  
データセットには、6,818のサンプルが含まれています。  
データセットにおける各特徴の説明は以下の通りです。

* country: 借り手の国
* en: 借り手の個人的なストーリー
* gender: 性別 (M=男性, F=女性)
* loan_amount: 承認された融資の金額
* nonpayment: 貸し手のタイプ（Lender＝Kivaに登録した個人ユーザー、Partner＝Kivaと協力するマイクロファイナンス機関)
* sector: 借り手の業種
* status: 融資のステータス(1-債務不履行, 0-返済)
 
今回は、enの列のみを使用します。  

https://www.kiva.org/ 

In [3]:
from pycaret.datasets import get_data

data = get_data("kiva")  # データの取得
data.shape  # データの形状

,country,en,gender,loan_amount,nonpayment,sector,status
0,Dominican Republic,"""Banco Esperanza"" is a group of 10 women looki...",F,1225,partner,Retail,0
1,Dominican Republic,"""Caminemos Hacia Adelante"" or ""Walking Forward...",F,1975,lender,Clothing,0
2,Dominican Republic,"""Creciendo Por La Union"" is a group of 10 peop...",F,2175,partner,Clothing,0
3,Dominican Republic,"""Cristo Vive"" (""Christ lives"" is a group of 10...",F,1425,partner,Clothing,0
4,Dominican Republic,"""Cristo Vive"" is a large group of 35 people, 2...",F,4025,partner,Food,0


(6818, 7)

## 環境の設定
PyCaretの環境を設定します。  
pycaret.nlpのsetup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

その際に、解析対象であるターゲット列を指定する必要があります。

In [4]:
from pycaret.nlp import setup

clf = setup(data=data, target="en", session_id=123)  # 環境の初期化

Description,Value
session_id,123
Documents,6818
Vocab Size,10671
Custom Stopwords,False


## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [5]:
from pycaret.nlp import models

models()  # 機械学習モデルの一覧

,Name,Reference
ID,,
lda,Latent Dirichlet Allocation,gensim/models/ldamodel
lsi,Latent Semantic Indexing,gensim/models/lsimodel
hdp,Hierarchical Dirichlet Process,gensim/models/hdpmodel
rp,Random Projections,gensim/models/rpmodel
nmf,Non-Negative Matrix Factorization,sklearn.decomposition.NMF



「トピックモデル」とは、機械学習や自然言語処理において、文書の集合体に出現する抽象的な「トピック」を発見するための統計モデルの一種です。

create_model関数は、モデルの訓練と評価を行います。  
以下のコードでは、トピックモデルの1つLatent Dirichlet Allocation (LDA)のモデルを作成し、訓練します。  
num_topicsでトピックの数を指定しますが、デフォルトのトピック数は4となります。  

In [6]:
from pycaret.nlp import create_model

lda = create_model("lda", num_topics=5, multi_core = True)  # LDAのモデルを作成し訓練

訓練済みモデルの概要を表示します。  

In [7]:
print(lda)

LdaModel(num_terms=10671, num_topics=5, decay=0.5, chunksize=100)


各ハイパーパラメータの意味は、pycaret.nlpの公式ドキュメントに解説があります。  
https://pycaret.readthedocs.io/en/latest/api/nlp.html#pycaret.nlp.create_model

## ラベルの割当て
`assign_model`関数により、各サンプルが各トピックに所属する割合を計算することができます。  

In [8]:
from pycaret.nlp import assign_model

lda_results = assign_model(lda)
lda_results.head()

,country,en,gender,loan_amount,nonpayment,sector,status,Topic_0,Topic_1,Topic_2,Topic_3,Topic_4,Dominant_Topic,Perc_Dominant_Topic
0,Dominican Republic,group woman look receive small loan take small...,F,1225,partner,Retail,0,0.987822,0.003048,0.003013,0.003053,0.003064,Topic 0,0.99
1,Dominican Republic,walk forward group entrepreneur seek second lo...,F,1975,lender,Clothing,0,0.840337,0.123801,0.002628,0.030539,0.002696,Topic 0,0.84
2,Dominican Republic,group people hope start business group look re...,F,2175,partner,Clothing,0,0.814926,0.004076,0.004031,0.004081,0.172886,Topic 0,0.81
3,Dominican Republic,live group woman look receive first loan young...,F,1425,partner,Clothing,0,0.711978,0.003120,0.003092,0.003129,0.278681,Topic 0,0.71
4,Dominican Republic,vive large group people hope take loan many se...,F,4025,partner,Food,0,0.669190,0.196513,0.003069,0.003107,0.128121,Topic 0,0.67


## 評価
plot_model関数により、様々な角度からコーパス全体、もしくは特定のトピックを分析することができます。  
https://pycaret.readthedocs.io/en/latest/api/nlp.html#pycaret.nlp.plot_model

plot_model関数で特にパラメータを指定しない場合、各単語の出現数がグラフで表示されます。

In [9]:
from pycaret.nlp import plot_model

plot_model()

plotに"bigram"を指定すると、連続した2単語の出現数がグラフで表示されます。

In [10]:
plot_model(plot="bigram")

topic_numにトピック名を指定することで、特定のトピックを解析することができます。  
以下のコードは、"Topic 1"という名前のトピックの、単語の出現数をグラフで表示します。

In [11]:
plot_model(lda, topic_num="Topic 1")

plotに"topic_distribution"を指定すると、各トッピックに分類されたサンプル数をグラフで確認することができます。  
各トピックの棒にカーソルを当てると、各トピックのキーワードが表示されます。

In [12]:
plot_model(lda, plot="topic_distribution")

t-SNE（T-Distributed Stochastic Neighbor Embedding ）は高次元データを低次元に変換して可視化するための次元削減アルゴリズムです。  
以下のコードのようにplotに"tsne"を指定すると、t-SNEによるデータの可視化が行われます。  
デフォルトでは3次元への圧縮となります。  

In [13]:
plot_model(lda, plot="tsne")